In [1]:
import os
import os.path
import numpy as np
import datetime
from datetime import timedelta 
import sys
sys.path.append("..")


try:
    import matplotlib.pyplot as plt
    %matplotlib inline
except:
    pass

try:
    import pandas as pd
    print("  pandas: %s"% pd.__version__)
except:
    print("Missing pandas package")
    

  pandas: 0.25.1


#### Part 1 : US Large cap index (Asset Annual Returns Calculation using daily prices)
**Instructions:**
We are using data from yahoo finance related to the US Large cap index
Calculate percent returns, also known as simple returns using asse_prices. assign the result to variable asset_returns. Keep only not-nan values in the resulting pandas.DataFrame


In [2]:
# load dataset
asset_prices = pd.read_csv('^DJUSL.csv',
                     date_parser=lambda dt: pd.to_datetime(dt, format='%Y-%m-%d'),
                     index_col = 0).dropna()

print('Asset prices shape', asset_prices.shape)
asset_prices.head(10)

Asset prices shape (2516, 6)


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2009-09-30,228.759995,229.360001,225.770004,228.000000,228.000000,10606500
2009-10-01,228.000000,228.000000,222.339996,222.440002,222.440002,9403100
2009-10-02,222.440002,222.679993,220.449997,221.580002,221.580002,7982900
2009-10-05,221.589996,225.089996,221.589996,224.520004,224.520004,6330100
2009-10-06,224.529999,228.820007,224.529999,227.639999,227.639999,7422100
2009-10-07,227.639999,228.429993,226.720001,228.350006,228.350006,6095800
2009-10-08,228.429993,231.029999,228.429993,229.869995,229.869995,7414900
2009-10-09,229.869995,231.149994,229.350006,231.139999,231.139999,5875500
2009-10-12,231.139999,232.910004,231.139999,232.190002,232.190002,5797600


In [3]:
asset_prices.tail(10)

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2019-09-16,637.200012,638.520020,635.940002,637.440002,637.440002,0
2019-09-17,636.820007,639.190002,636.510010,639.090027,639.090027,0
2019-09-18,638.130005,639.820007,633.619995,639.539978,639.539978,0
2019-09-19,640.469971,643.130005,639.250000,640.070007,640.070007,0
2019-09-20,640.650024,642.119995,635.059998,636.659973,636.659973,0
2019-09-23,634.929993,637.969971,634.640015,636.429993,636.429993,0
2019-09-24,638.659973,640.099976,629.020020,630.830017,630.830017,0
2019-09-25,631.309998,635.859985,627.950012,634.820007,634.820007,0
2019-09-26,635.059998,635.489990,630.289978,633.369995,633.369995,0


Now we will compute the monthly return using pct_change method

In [4]:
asset_daily_returns=asset_prices["Adj Close"].pct_change() 
asset_daily_returns=pd.DataFrame(asset_daily_returns)
asset_daily_returns.rename(columns={'Adj Close':'monthly returns'}, inplace=True)
asset_daily_returns

,monthly returns
Date,
2009-09-30,NaN
2009-10-01,-0.024386
2009-10-02,-0.003866
2009-10-05,0.013268
2009-10-06,0.013896
...,...
2019-09-23,-0.000361
2019-09-24,-0.008799
2019-09-25,0.006325


In [5]:
asset_daily_returns.dropna()

,monthly returns
Date,
2009-10-01,-0.024386
2009-10-02,-0.003866
2009-10-05,0.013268
2009-10-06,0.013896
2009-10-07,0.003119
...,...
2019-09-23,-0.000361
2019-09-24,-0.008799
2019-09-25,0.006325


In [6]:
def sharpe_ratio(ts_returns, periods_per_year=252):
    """
    sharpe_ratio - Calculates annualized return, annualized vol, and annualized sharpe ratio, 
                    where sharpe ratio is defined as annualized return divided by annualized volatility 
                    
    Arguments:
    ts_returns - pd.Series of returns of a single eigen portfolio
    
    Return:
    a tuple of three doubles: annualized return, volatility, and sharpe ratio
    """
    
    annualized_return = 0.
    annualized_vol = 0.
    annualized_sharpe = 0.
    
  
    n = len(ts_returns)/periods_per_year
    annualized_return = np.prod(1+ts_returns)**(1/n)-1
    annualized_vol = ts_returns.std()* np.sqrt(periods_per_year)
    annualized_sharpe=annualized_return / annualized_vol

    
    return annualized_return, annualized_vol, annualized_sharpe

In [7]:
annualized_return=[]
annualized_vol=[]
annualized_sharpe=[]

step1 : calculating annualized_return, annualized_vol and annualized_sharpe for the period 2009_2019 and storing them in different python lists

In [8]:
_return, _vol, _sharpe = sharpe_ratio(asset_daily_returns.dropna(), periods_per_year=252)
annualized_return.append(_return[0])
annualized_vol.append(_vol[0])
annualized_sharpe.append(_sharpe[0])

Step2: calculating the yearly return , volatility and sharpe ration  from 2009 to 2019

In [9]:
from datetime import date

dt_start = np.datetime64('2019-01-01')
dt_end = np.datetime64('2019-12-31')


for i in range (11):
    flag_year = np.logical_and(asset_daily_returns.index > dt_start, asset_daily_returns.index < dt_end)
    asset_daily_returns_oneyear=asset_daily_returns.loc[flag_year]
    _return, _vol, _sharpe = sharpe_ratio(asset_daily_returns_oneyear.dropna(), periods_per_year=252)
    annualized_return.append(_return[0])
    annualized_vol.append(_vol[0])
    annualized_sharpe.append(_sharpe[0])
    dt_start = dt_start.astype(object) 
    dt_end = dt_end.astype(object) 
    dt_start = np.datetime64(dt_start.replace(dt_start.year-1))
    dt_end = np.datetime64(dt_end.replace(dt_end.year-1))
    
    

    
    

In [10]:
index=['2009-2019','2019','2018','2017','2016','2015','2014','2013','2012','2011','2010','2009']

In [13]:
 statistics = pd.DataFrame(data ={'Annual return': annualized_return ,'Annual volatility': annualized_vol ,
                                  'sharpe ratio': annualized_sharpe}, index =index)

In [14]:
statistics

,Annual return,Annual volatility,sharpe ratio
2009-2019,0.107171,0.147805,0.725080
2019,0.244185,0.134832,1.811035
2018,-0.059570,0.174099,-0.342161
2017,0.201667,0.066143,3.048978
2016,0.090641,0.129458,0.700162
2015,0.004148,0.155665,0.026646
2014,0.123023,0.112773,1.090899
2013,0.286551,0.109477,2.617451
2012,0.115383,0.124989,0.923147
2011,-0.000599,0.228545,-0.002622
